In [1]:
!pip install fastapi uvicorn pandas requests nest-asyncio pyngrok


   ---------------------------------------- 0.0/95.6 kB ? eta -:--:--
   ---------------------------------------- 95.6/95.6 kB 2.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/66.4 kB ? eta -:--:--
   ---------------------------------------- 66.4/66.4 kB 3.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/73.0 kB ? eta -:--:--
   ---------------------------------------- 73.0/73.0 kB 4.2 MB/s eta 0:00:00


In [13]:
import os, requests, pandas as pd
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv("TOMTOM_API_KEY")
assert API_KEY, "No TOMTOM_API_KEY found. Put it in a .env file first."

NYC_POINTS = [
    {"name": "Times Square",        "lat": 40.7580, "lng": -73.9855},
    {"name": "Central Park South",  "lat": 40.7651, "lng": -73.9769},
    {"name": "Grand Central",       "lat": 40.7527, "lng": -73.9772},
    {"name": "Wall Street",         "lat": 40.7066, "lng": -74.0090},
    {"name": "Brooklyn Bridge",     "lat": 40.7061, "lng": -73.9969},
]


In [15]:
def get_flow(lat, lng):
    url = (
        "https://api.tomtom.com/traffic/services/4/"
        f"flowSegmentData/relative0/10/json?point={lat},{lng}&key={API_KEY}"
    )
    return requests.get(url, timeout=10).json().get("flowSegmentData")  # dict or None


In [45]:
rows = []
for p in NYC_POINTS:  # <- use NYC_POINTS (not POINTS)
    d = get_flow(p["lat"], p["lng"])
    if not d:
        print(f"No data for {p['name']}")
        continue
    rows.append({
        "name": p["name"],
        "lat": p["lat"],
        "lng": p["lng"],
        "currentSpeed": d.get("currentSpeed"),
        "freeFlowSpeed": d.get("freeFlowSpeed"),
        "currentTravelTime": d.get("currentTravelTime"),
        "freeFlowTravelTime": d.get("freeFlowTravelTime"),
        "confidence": d.get("confidence"),
    })

df = pd.DataFrame(rows)
df


,name,lat,lng,currentSpeed,freeFlowSpeed,currentTravelTime,freeFlowTravelTime,confidence
0,Times Square,40.7580,-73.9855,11,11,130,130,1
1,Central Park South,40.7651,-73.9769,11,11,147,147,1
2,Grand Central,40.7527,-73.9772,13,13,317,317,1
3,Wall Street,40.7066,-74.0090,18,18,180,180,1
4,Brooklyn Bridge,40.7061,-73.9969,44,44,193,193,1


In [23]:
df["congestion_ratio"] = df["currentSpeed"] / df["freeFlowSpeed"]
df


,name,currentSpeed,freeFlowSpeed,congestion_ratio
0,Times Square,11,11,1.000000
1,Central Park South,11,11,1.000000
2,Grand Central,13,13,1.000000
3,Wall Street,14,17,0.823529
4,Brooklyn Bridge,44,44,1.000000


In [25]:
!pip install folium


In [47]:
import folium

m = folium.Map(location=[40.733, -73.995], zoom_start=12)

for _, r in df.iterrows():
    folium.Marker(
        location=[r["lat"], r["lng"]],
        popup=f"{r['name']}: {r['currentSpeed']} (free {r['freeFlowSpeed']})",
    ).add_to(m)

m  # shows in notebook
